From Doug Turnbull's [Real life NDCG notebook](https://softwaredoug.com/blog/2024/10/19/real-life-ndcg)

## Real-life NDCG computation

You replay a set of search queries to your system, and concat them together into one big dataframe of results.

Off to the side, you have labels for documents for these queries.

Let's create some simple search relevance evaluation functions, taking into account the typical decisions you need to make when evaluating offline (missing labels, choosing the right ideal)

### Input 1 - Labeled judgments

Here's a labels (ie [judgments](https://softwaredoug.com/blog/2021/02/21/what-is-a-judgment-list)) dataframe with two queries.

Let's pretend we take these queries and replay them to our test search system, we get back the results in the subsequent cell

In [1]:
import pandas as pd

labels_df = pd.DataFrame(
    [
        {"query_id": 1, "query": "blue shoes", "grade": 0.9, "doc_id": 125125},
        {"query_id": 1, "query": "blue shoes", "grade": 0.9, "doc_id": 5678},
        {"query_id": 1, "query": "blue shoes", "grade": 0.1, "doc_id": 1122},
        {"query_id": 2, "query": "red shoes", "grade": 1.0, "doc_id": 12225},
        {"query_id": 2, "query": "red shoes", "grade": 0.9, "doc_id": 1521},
        {"query_id": 2, "query": "red shoes", "grade": 0.8, "doc_id": 5125},
        {"query_id": 2, "query": "red shoes", "grade": 0.1, "doc_id": 1111},
    ]
)

labels_df

,query_id,query,grade,doc_id
0,1,blue shoes,0.9,125125
1,1,blue shoes,0.9,5678
2,1,blue shoes,0.1,1122
3,2,red shoes,1.0,12225
4,2,red shoes,0.9,1521
5,2,red shoes,0.8,5125
6,2,red shoes,0.1,1111


### Input 2 - replay results from the search engine

We issued our two queries, and concatted their results to a single dataframe.

In [2]:
results_df = pd.DataFrame(
    [
        {"query_id": 1, "rank": 1, "query": "blue shoes", "doc_id": 5678},
        {"query_id": 1, "rank": 2, "query": "blue shoes", "doc_id": 1122},
        {"query_id": 2, "rank": 1, "query": "red shoes", "doc_id": 1521},
        {"query_id": 2, "rank": 2, "query": "red shoes", "doc_id": 1251},
        {"query_id": 2, "rank": 3, "query": "red shoes", "doc_id": 5125},
    ]
)
results_df

,query_id,rank,query,doc_id
0,1,1,blue shoes,5678
1,1,2,blue shoes,1122
2,2,1,red shoes,1521
3,2,2,red shoes,1251
4,2,3,red shoes,5125


### Label results

We now need to assign labels to our search results given the label dataframe. That's easy enough to do by joining labels into the results.

In [3]:
labeled = results_df.merge(labels_df, how="left", on=["query_id", "query", "doc_id"])
labeled

,query_id,rank,query,doc_id,grade
0,1,1,blue shoes,5678,0.9
1,1,2,blue shoes,1122,0.1
2,2,1,red shoes,1521,0.9
3,2,2,red shoes,1251,NaN
4,2,3,red shoes,5125,0.8


## Add in the basic DCG stats

* gain - basically the grade, but very strongly biased to high grades (2^ grade) - 1
* discount - position discount. Getting higher ranks better matters more.
* discounted gain - gain * discount

In [4]:
import numpy as np

labeled["gain"] = (2 ** labeled["grade"]) - 1
labeled["discount"] = 1 / (np.log(labeled["rank"] + 1))
labeled["discounted_gain"] = labeled["gain"] * labeled["discount"]
labeled

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469
3,2,2,red shoes,1251,NaN,NaN,0.910239,NaN
4,2,3,red shoes,5125,0.8,0.741101,0.721348,0.534591


## Decision 1 - handling unlabeled results?

How do we handle cases where we have to grade? The two most common options are:

* FILTER - aka DCG-f - we remove the unlabeled result, and pretend the search engine only returned the labeled ones
* ZERO - aka DCG-z - we set it to zero and treat the unlabeled result as irrelevant


### DCG-z - set unlabeled as 0

For now, let's do DCG-z

In [5]:
labeled_z = labeled.fillna({"discounted_gain": 0})
labeled_z

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469
3,2,2,red shoes,1251,NaN,NaN,0.910239,0.000000
4,2,3,red shoes,5125,0.8,0.741101,0.721348,0.534591


In [6]:
dcg_zs = labeled_z.groupby("query_id")["discounted_gain"].sum()
dcg_zs

query_id
1    1.314800
2    1.784061
Name: discounted_gain, dtype: float64

### DCG-f - ignore unlabeled

In [7]:
labeled_f = labeled[~labeled["discounted_gain"].isna()]
labeled_f

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469
4,2,3,red shoes,5125,0.8,0.741101,0.721348,0.534591


#### Adjust gain after filtering

In [8]:
labeled_f["rank"] = labeled_f.groupby("query_id").cumcount() + 1
labeled_f["discount"] = 1 / (np.log(labeled_f["rank"] + 1))
labeled_f["discounted_gain"] = labeled_f["gain"] * labeled_f["discount"]

labeled_f

/var/folders/t3/6j10m6_j77bb7jpj04jbthb80000gn/T/ipykernel_12672/2627131693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_f["rank"] = labeled_f.groupby("query_id").cumcount() + 1
/var/folders/t3/6j10m6_j77bb7jpj04jbthb80000gn/T/ipykernel_12672/2627131693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_f["discount"] = 1 / (np.log(labeled_f["rank"] + 1))
/var/folders/t3/6j10m6_j77bb7jpj04jbthb80000gn/T/ipykernel_12672/2627131693.py:3: SettingWithCopyWarning: 
A value is trying to be 

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469
4,2,2,red shoes,5125,0.8,0.741101,0.910239,0.674579


In [9]:
dcg_f = labeled_f.groupby("query_id")["discounted_gain"].sum()
dcg_f

query_id
1    1.314800
2    1.924048
Name: discounted_gain, dtype: float64

## Decision 2 - Compute normalized DCG, but to which ideal?

*See blog post [Flavors of NDCG](https://softwaredoug.com/blog/2024/05/22/flavors-of-ndcg)*

To normalize DCG (ie compute NDCG = DCG / ideal DCG), we need to compute the ideal DCG. But there's several ways to compute the ideal:

1. Local - NDCG-l - just the ideal ranking over the current returned search results. Purely measuring precision within the result set.
2. Global - NDCG-g - the ideal over all the labels, which helps also measure our recall relative to the known ideal results.
3. Max - NDCG-m - the ideal is the max label put in each position - measuring recall + whether our system even has relevant content for this query.

Local causes NDCG to look more optimistic, max the most pessimistic.

### NDCG relative to local iDCG

Local == take labeled search results, sort by grade, compute ideal DCG within taht window.

Local depends on whether you're filtering or assigning to zero. Let's just grab the filter DF. Then compute a DCG as if this was sorted by the labeled.

Then below we compute an ideal rank + DCG stats, finally an iDCG-l

In [10]:
ideal_results = labeled_f.sort_values(["query_id", "grade"], ascending=[True, False])
ideal_results

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469
4,2,2,red shoes,5125,0.8,0.741101,0.910239,0.674579


In [11]:
labeled_lf = labeled_f.copy()
labeled_lf.loc[:, "ideal_rank"] = ideal_results.groupby("query_id").cumcount() + 1
labeled_lf

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain,ideal_rank
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469,1
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331,2
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469,1
4,2,2,red shoes,5125,0.8,0.741101,0.910239,0.674579,2


In [12]:
labeled_lf.loc[:, "ideal_discount"] = 1 / (np.log(labeled_lf["ideal_rank"] + 1))
labeled_lf.loc[:, "ideal_discounted_gain"] = labeled_lf["ideal_discount"] * labeled_lf["gain"]
labeled_lf

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain,ideal_rank,ideal_discount,ideal_discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469,1,1.442695,1.249469
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331,2,0.910239,0.065331
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469,1,1.442695,1.249469
4,2,2,red shoes,5125,0.8,0.741101,0.910239,0.674579,2,0.910239,0.674579


In [13]:
idcg_lf = labeled_lf.groupby("query_id")["ideal_discounted_gain"].sum()
idcg_lf

query_id
1    1.314800
2    1.924048
Name: ideal_discounted_gain, dtype: float64

In [14]:
ndcg_lf = dcg_f / idcg_lf
ndcg_lf

query_id
1    1.0
2    1.0
dtype: float64

### NDCG relative to global ideal

Let's compute an IDCG from the labels, as if they were sorted to give the best results, to evaluate our DCG relative to a search engine that could retrieve these results in the top N.

You'll notice this stat is considerably worse

In [15]:
labels_df_ideal = labels_df.sort_values(["query_id", "grade"], ascending=[True, False])
labels_df_ideal.loc[:, "ideal_rank"] = labels_df_ideal.groupby("query_id").cumcount() + 1
labels_df_ideal.loc[:, "gain"] = (2 ** labels_df_ideal["grade"]) - 1
labels_df_ideal.loc[:, "discount"] = 1 / (np.log(labels_df_ideal["ideal_rank"] + 1))
labels_df_ideal.loc[:, "discounted_gain"] = labels_df_ideal["discount"] * labels_df_ideal["gain"]

labels_df_ideal

,query_id,query,grade,doc_id,ideal_rank,gain,discount,discounted_gain
0,1,blue shoes,0.9,125125,1,0.866066,1.442695,1.249469
1,1,blue shoes,0.9,5678,2,0.866066,0.910239,0.788327
2,1,blue shoes,0.1,1122,3,0.071773,0.721348,0.051774
3,2,red shoes,1.0,12225,1,1.000000,1.442695,1.442695
4,2,red shoes,0.9,1521,2,0.866066,0.910239,0.788327
5,2,red shoes,0.8,5125,3,0.741101,0.721348,0.534591
6,2,red shoes,0.1,1111,4,0.071773,0.621335,0.044595


In [16]:
idcg_g = labels_df_ideal.groupby("query_id")["discounted_gain"].sum()
idcg_g

query_id
1    2.089570
2    2.810209
Name: discounted_gain, dtype: float64

In [17]:
ndcg_fg = dcg_f / idcg_g
ndcg_fg

query_id
1    0.629220
2    0.684664
Name: discounted_gain, dtype: float64

### NDCG relative to max ideal

Let's compute an IDCG the most pesimistically possible - as if the max label was used on every result.

In [18]:
MAX_LABEL = 1.0

max_gain = (2**MAX_LABEL) - 1

labeled_mf = labeled_f.copy()

labeled_mf.loc[:, "ideal_discounted_gain"] = labeled_mf["discount"] * max_gain
labeled_mf

,query_id,rank,query,doc_id,grade,gain,discount,discounted_gain,ideal_discounted_gain
0,1,1,blue shoes,5678,0.9,0.866066,1.442695,1.249469,1.442695
1,1,2,blue shoes,1122,0.1,0.071773,0.910239,0.065331,0.910239
2,2,1,red shoes,1521,0.9,0.866066,1.442695,1.249469,1.442695
4,2,2,red shoes,5125,0.8,0.741101,0.910239,0.674579,0.910239


In [19]:
idcg_fm = labeled_mf.groupby("query_id")["ideal_discounted_gain"].sum()
idcg_fm

query_id
1    2.352934
2    2.352934
Name: ideal_discounted_gain, dtype: float64

In [20]:
ndcg_fm = dcg_f / idcg_fm
ndcg_fm

query_id
1    0.558792
2    0.817723
dtype: float64

## DCG / NDCG up to what position?

We often say 'DCG@N' as in 'DCG@10' to talk about the DCG, but only computed. from the top 10 results. So far we have used the full retrieved result set, and ignored this.

But this is easy to adjust :)

Below just make sure filter our DCG / ideal DCGs to only sum up to a given rank

In [21]:
AT = 10

MAX_LABEL = 1.0

max_gain = (2**MAX_LABEL) - 1

idcg_m_10 = 0
for i in range(1, AT + 1):
    discount = 1 / (np.log(i + 1))
    idcg_m_10 += max_gain * discount

idcg_m_10

np.float64(6.554970525044798)

In [22]:
dcg_fm_10 = labeled_f[labeled_f["rank"] < AT].groupby("query_id")["discounted_gain"].sum()
dcg_fm_10

query_id
1    1.314800
2    1.924048
Name: discounted_gain, dtype: float64

In [23]:
ndcg_fm_10 = dcg_fm_10 / idcg_m_10
ndcg_fm_10

query_id
1    0.200581
2    0.293525
Name: discounted_gain, dtype: float64

## Jaccard diff result sets

In [24]:
results_df2 = pd.DataFrame(
    [
        {"query_id": 1, "rank": 1, "query": "blue shoes", "doc_id": 5678},
        {"query_id": 1, "rank": 2, "query": "blue shoes", "doc_id": 2511},
        {"query_id": 2, "rank": 1, "query": "red shoes", "doc_id": 1521},
        {"query_id": 2, "rank": 2, "query": "red shoes", "doc_id": 1251},
        {"query_id": 2, "rank": 3, "query": "red shoes", "doc_id": 5125},
    ]
)
results_df2

jaccard_df = (
    results_df.groupby(["query_id", "query"])["doc_id"].agg(set).to_frame().rename(columns={"doc_id": "results1"})
)
jaccard_df

,,results1
query_id,query,
1,blue shoes,"{1122, 5678}"
2,red shoes,"{1521, 1251, 5125}"


In [25]:
jaccard_df["results2"] = results_df2.groupby(["query_id", "query"])["doc_id"].agg(set)
jaccard_df["intersection"] = jaccard_df.apply(lambda row: (row["results1"] & row["results2"]), axis=1)
jaccard_df["union"] = jaccard_df.apply(lambda row: (row["results1"] | row["results2"]), axis=1)
jaccard_df["jaccard"] = jaccard_df["intersection"].apply(len) / jaccard_df["union"].apply(len)

jaccard_df

,,results1,results2,intersection,union,jaccard
query_id,query,,,,,
1,blue shoes,"{1122, 5678}","{5678, 2511}",{5678},"{1122, 5678, 2511}",0.333333
2,red shoes,"{1521, 1251, 5125}","{1521, 1251, 5125}","{1521, 1251, 5125}","{1521, 1251, 5125}",1.000000
